<a href="https://colab.research.google.com/github/kelliebanzon/slo-hacks-2019/blob/master/playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Setup**

In [1]:
!pip install spotipy


  Stored in directory: /root/.cache/pip/wheels/76/28/19/a86ca9bb0e32dbd4a4f580870250f5aeef852870578e0427e6
Successfully built spotipy


In [0]:
import sys
import json
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

**Authentication**

In [0]:
username = "INSERT-USERNAME"
scope = "user-library-read

client_credentials_manager = SpotifyClientCredentials(client_id='c2cf260ff28e4694bcb1eb0a29191b74', client_secret='83713ae562c04d7e96220f60749a6989')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

**Loop though users playlists**


In [10]:
playlists = sp.user_playlists(username)

for playlist in playlists['items']: 
    if playlist['owner']['id'] == username:
        print(playlist['name'])
        print('  total tracks', playlist['tracks']['total'])
        
        # get playlist data
        results = sp.user_playlist(username, playlist['id'], fields="fields=tracks")
        tracks_list = results['tracks']["items"]
        tracks_list_short = tracks_list[0:20]
        
        # prep lists of tracks
        tracks = []
        for track_obj in tracks_list_short:
          tracks.append(track_obj["track"]['id'])
        
        # get features batch of tracks
        features = sp.audio_features(tracks)
        print(json.dumps(features))


test
  total tracks 11
[{"danceability": 0.419, "energy": 0.519, "key": 5, "loudness": -6.47, "mode": 1, "speechiness": 0.0334, "acousticness": 0.11, "instrumentalness": 0.889, "liveness": 0.409, "valence": 0.343, "tempo": 94.044, "type": "audio_features", "id": "17Nowmq4iF2rkbd1rAe1Vt", "uri": "spotify:track:17Nowmq4iF2rkbd1rAe1Vt", "track_href": "https://api.spotify.com/v1/tracks/17Nowmq4iF2rkbd1rAe1Vt", "analysis_url": "https://api.spotify.com/v1/audio-analysis/17Nowmq4iF2rkbd1rAe1Vt", "duration_ms": 120427, "time_signature": 4}, {"danceability": 0.221, "energy": 0.826, "key": 5, "loudness": -5.393, "mode": 1, "speechiness": 0.126, "acousticness": 0.0514, "instrumentalness": 0, "liveness": 0.135, "valence": 0.116, "tempo": 130.632, "type": "audio_features", "id": "6dFlPqbcAjpvDKp8voczqX", "uri": "spotify:track:6dFlPqbcAjpvDKp8voczqX", "track_href": "https://api.spotify.com/v1/tracks/6dFlPqbcAjpvDKp8voczqX", "analysis_url": "https://api.spotify.com/v1/audio-analysis/6dFlPqbcAjpvDKp8v

In [0]:
# TODO
# 1. get tracks for users playlist
# 2. splits into track_list into batches of 25


# just incase we really needed these...
# util.prompt_for_user_token(username,scope,client_id='c2cf260ff28e4694bcb1eb0a29191b74',client_secret='83713ae562c04d7e96220f60749a6989',redirect_uri='http://localhost:8080')
# TOKEN = util.prompt_for_user_token(username, scope)
